In [1]:
import numpy as np
import emoji,csv
import matplotlib.pyplot as plt
%matplotlib inline


In [4]:
def read_csv(filename = 'data/emojify_data.csv'):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            emoji.append(row[1])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)

    return X, Y

X_train,Y_train = read_csv("/home/yash/DeepLearning/data/train_emoji.csv")
X_test,Y_test = read_csv("/home/yash/DeepLearning/data/tesss.csv")
print(X_train.shape)
print(Y_test.shape)

(132,)
(56,)


In [6]:
maxLen = len(max(X_train, key=lambda x: len(x.split())).split())
print(maxLen)

10


In [9]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

def label_to_emoji(label):
    """
    Converts a label (int or string) into the corresponding emoji code (string) ready to be printed
    """
    return emoji.emojize(emoji_dictionary[str(label)])

for idx in range(10):
    print(X_train[idx], label_to_emoji(Y_train[idx]))

never talk to me again :disappointed:
I am proud of your achievements :smile:
It is the worst day in my life :disappointed:
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit :disappointed:
congratulations on your acceptance :smile:
The assignment is too long  :disappointed:
I want to go play ⚾


In [ ]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)
print(Y_oh_test)

In [14]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

w_to_i, i_to_w, word_map = read_glove_vecs('/home/yash/DeepLearning/data/glove.6B.50d.txt')

In [15]:
len(word_map.keys())

400000

In [17]:
def  sentence_to_avg(sentence: str, word_map: dict) :
    any_word = next(iter(word_map.keys()))

    words = sentence.lower().split()

    avg = np.zeros(word_map[any_word].shape)
    count = 0

    for w in words:
        if w in word_map:
            avg += word_map[w]
            count += 1

    if count > 0:
        avg /= count
    
    return avg

sentence = "I am so happy to be here"
print(len(sentence_to_avg(sentence, word_map)))


50


## Now we have all the important parts required to build the model function()

In [27]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def predict(X, Y, W, b, word_to_vec_map):
    """
    Given X (sentences) and Y (emoji indices), predict emojis and compute the accuracy of your model over the given set.
    
    Arguments:
    X -- input data containing sentences, numpy array of shape (m, None)
    Y -- labels, containing index of the label emoji, numpy array of shape (m, 1)
    
    Returns:
    pred -- numpy array of shape (m, 1) with your predictions
    """
    m = X.shape[0]
    pred = np.zeros((m, 1))
    any_word = list(word_to_vec_map.keys())[0]
    # number of classes  
    n_h = word_to_vec_map[any_word].shape[0] 
    
    for j in range(m):                       # Loop over training examples
        
        # Split jth test example (sentence) into list of lower case words
        words = X[j].lower().split()
        
        # Average words' vectors
        avg = np.zeros((n_h,))
        count = 0
        for w in words:
            if w in word_to_vec_map:
                avg += word_to_vec_map[w]
                count += 1
        
        if count > 0:
            avg = avg / count

        # Forward propagation
        Z = np.dot(W, avg) + b
        A = softmax(Z)
        pred[j] = np.argmax(A)
        
    print("Accuracy: "  + str(np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:]))))
    
    return pred


def model(X:str,Y:int,word_map: dict, learning_rate= 0.01, num_iterations = 500):
    any_word = next(iter(word_map.keys()))

    m = Y.shape[0]
    n_y = len(np.unique(Y)) # No of classes
    n_h = word_map[any_word].shape[0] # Dimensions of Glove vectors

    #Initialize the weights using Xavier initialization
    W = np.random.randn(n_y,n_h) / np.sqrt(n_h)
    b = np.zeros(n_y)

    Y_oh = convert_to_one_hot(Y,n_y)

    for t in range(num_iterations):
        cost = 0
        dW = 0
        db = 0

        for i in range(m):
            avg = sentence_to_avg(X[i], word_map)

            # Forward propagate the avg through the softmax layer
            z = np.add(np.dot(W,avg),b)
            a = softmax(z)

            # Compute cost using the i'th training label's one hot representation and "A" (the output of the softmax)
            cost = -np.sum(np.dot(Y_oh[i], np.log(a)))
            ### END CODE HERE ###
            
            # Compute gradients 
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz

            # Update parameters with Stochastic Gradient Descent
            W = W - learning_rate * dW
            b = b - learning_rate * db
            
        if t % 100 == 0:
            print(f"Epoch {t} has a cost of {cost}")
            pred = predict(X,Y,W,b,word_map)

    return pred, W ,b

In [28]:
pred , W, b = model(X_train,Y_train,word_map)
# print(pred)

Epoch 0 has a cost of 1.6954219635708256
Accuracy: 0.36363636363636365
Epoch 100 has a cost of 0.0638715412685588
Accuracy: 0.9242424242424242
Epoch 200 has a cost of 0.03879139080755904
Accuracy: 0.9621212121212122
Epoch 300 has a cost of 0.031394490231089145
Accuracy: 0.9621212121212122
Epoch 400 has a cost of 0.027338111946778487
Accuracy: 0.9772727272727273


In [29]:
# Predicting on train and test
print("Train set:")
pred_train = predict(X_train,Y_train,W,b,word_map)
print(pred_train)

print("Test set:")
pred_test = predict(X_test,Y_test,W,b,word_map)
print(pred_test)

Train set:
Accuracy: 0.9772727272727273
[[3.]
 [2.]
 [3.]
 [0.]
 [4.]
 [0.]
 [3.]
 [2.]
 [3.]
 [1.]
 [3.]
 [3.]
 [1.]
 [3.]
 [2.]
 [3.]
 [2.]
 [3.]
 [1.]
 [2.]
 [3.]
 [0.]
 [2.]
 [2.]
 [2.]
 [1.]
 [4.]
 [3.]
 [3.]
 [4.]
 [0.]
 [3.]
 [4.]
 [2.]
 [0.]
 [3.]
 [2.]
 [2.]
 [3.]
 [4.]
 [2.]
 [2.]
 [0.]
 [2.]
 [3.]
 [0.]
 [3.]
 [2.]
 [4.]
 [3.]
 [0.]
 [3.]
 [3.]
 [3.]
 [4.]
 [2.]
 [1.]
 [1.]
 [1.]
 [2.]
 [3.]
 [1.]
 [0.]
 [0.]
 [0.]
 [3.]
 [4.]
 [4.]
 [2.]
 [2.]
 [1.]
 [2.]
 [0.]
 [3.]
 [2.]
 [2.]
 [0.]
 [0.]
 [3.]
 [1.]
 [2.]
 [1.]
 [2.]
 [2.]
 [4.]
 [3.]
 [3.]
 [2.]
 [4.]
 [0.]
 [0.]
 [3.]
 [3.]
 [3.]
 [3.]
 [2.]
 [0.]
 [1.]
 [2.]
 [3.]
 [0.]
 [2.]
 [2.]
 [2.]
 [3.]
 [2.]
 [2.]
 [2.]
 [4.]
 [1.]
 [1.]
 [3.]
 [3.]
 [4.]
 [1.]
 [2.]
 [1.]
 [1.]
 [3.]
 [1.]
 [0.]
 [4.]
 [0.]
 [3.]
 [3.]
 [4.]
 [4.]
 [1.]
 [4.]
 [3.]
 [0.]
 [2.]]
Test set:
Accuracy: 0.875
[[4.]
 [3.]
 [2.]
 [2.]
 [2.]
 [2.]
 [3.]
 [2.]
 [4.]
 [2.]
 [1.]
 [2.]
 [0.]
 [3.]
 [1.]
 [3.]
 [2.]
 [2.]
 [3.]
 [4.]
 [3.]
 [2.]
 [4.]
 [3

In [30]:
X_my_sentences = np.array(["i treasure you", "i love you", "funny lol", "lets play with a ball", "food is ready", "today is not good"])
Y_my_labels = np.array([[0], [0], [2], [1], [4],[3]])

pred = predict(X_my_sentences, Y_my_labels , W, b, word_map)
print(pred)

Accuracy: 0.8333333333333334
[[0.]
 [0.]
 [2.]
 [1.]
 [4.]
 [2.]]
